<a href="https://colab.research.google.com/github/james130625/notebook/blob/main/%ED%8C%A9%ED%84%B0_%EB%AA%A8%EB%8D%B8%EB%A7%81_Factor_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install yfinance
!pip install pandas
!pip install numpy
!pip install statsmodels

import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import zipfile
import io
import requests
from datetime import datetime

# 오늘 날짜 가져오기
today = datetime.today().strftime('%Y-%m-%d')

# 주식 데이터 다운로드 (예: 애플 주식)
stock_data = yf.download('AAPL', start='2020-01-01', end=today)

# 시장 데이터 다운로드 (예: S&P 500 지수)
market_data = yf.download('^GSPC', start='2020-01-01', end=today)

# 팩터 데이터 다운로드 (Fama-French 3-팩터)
url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip'
response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open('F-F_Research_Data_Factors_daily.CSV') as f:
        ff_factors = pd.read_csv(f, skiprows=3)

# 데이터 전처리
stock_data['Daily Return'] = stock_data['Adj Close'].pct_change()
market_data['Daily Return'] = market_data['Adj Close'].pct_change()
ff_factors = ff_factors.iloc[:-1]
ff_factors['Date'] = pd.to_datetime(ff_factors['Unnamed: 0'], format='%Y%m%d')
ff_factors.set_index('Date', inplace=True)
ff_factors = ff_factors[['Mkt-RF', 'SMB', 'HML', 'RF']].astype(float)

# 결측값 및 무한대 값 제거
stock_data = stock_data.dropna()
market_data = market_data.dropna()
ff_factors = ff_factors.replace([np.inf, -np.inf], np.nan).dropna()

# 공통 날짜 인덱스 설정
common_index = stock_data.index.intersection(market_data.index).intersection(ff_factors.index)
stock_data = stock_data.loc[common_index]
market_data = market_data.loc[common_index]
ff_factors = ff_factors.loc[common_index]

# CAPM: 주식 수익률과 시장 수익률 간의 회귀 분석
X_capm = market_data['Daily Return']
y_capm = stock_data['Daily Return'] - ff_factors['RF'] / 100

X_capm = sm.add_constant(X_capm)
capm_model = sm.OLS(y_capm, X_capm).fit()
print(capm_model.summary())

# Fama-French 3-팩터 모델: 주식 수익률과 팩터 수익률 간의 회귀 분석
ff_data = ff_factors.join(stock_data[['Daily Return']])
ff_data = ff_data.dropna()
X_ff = ff_data[['Mkt-RF', 'SMB', 'HML']]
y_ff = ff_data['Daily Return'] - ff_factors['RF'] / 100

X_ff = sm.add_constant(X_ff)
ff_model = sm.OLS(y_ff, X_ff).fit()
print(ff_model.summary())


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.650
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     2014.
Date:                Wed, 12 Jun 2024   Prob (F-statistic):          1.33e-249
Time:                        02:21:46   Log-Likelihood:                 3247.3
No. Observations:                1088   AIC:                            -6491.
Df Residuals:                    1086   BIC:                            -6481.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0003      0.000      0.870   

In [5]:
!pip install yfinance
!pip install pandas
!pip install numpy
!pip install statsmodels

import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import zipfile
import io
import requests
from datetime import datetime

# 오늘 날짜 가져오기
today = datetime.today().strftime('%Y-%m-%d')

# 삼성전자 데이터 다운로드
stock_data = yf.download('005930.KS', start='2020-01-01', end=today)

# 코스피 지수 데이터 다운로드
market_data = yf.download('^KS11', start='2020-01-01', end=today)

# 팩터 데이터 다운로드 (Fama-French 3-팩터, 한국 데이터는 없기 때문에 글로벌 데이터 사용)
url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip'
response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open('F-F_Research_Data_Factors_daily.CSV') as f:
        ff_factors = pd.read_csv(f, skiprows=3)

# 데이터 전처리
stock_data['Daily Return'] = stock_data['Adj Close'].pct_change()
market_data['Daily Return'] = market_data['Adj Close'].pct_change()
ff_factors = ff_factors.iloc[:-1]
ff_factors['Date'] = pd.to_datetime(ff_factors['Unnamed: 0'], format='%Y%m%d')
ff_factors.set_index('Date', inplace=True)
ff_factors = ff_factors[['Mkt-RF', 'SMB', 'HML', 'RF']].astype(float)

# 결측값 및 무한대 값 제거
stock_data = stock_data.dropna()
market_data = market_data.dropna()
ff_factors = ff_factors.replace([np.inf, -np.inf], np.nan).dropna()

# 공통 날짜 인덱스 설정
common_index = stock_data.index.intersection(market_data.index).intersection(ff_factors.index)
stock_data = stock_data.loc[common_index]
market_data = market_data.loc[common_index]
ff_factors = ff_factors.loc[common_index]

# CAPM: 주식 수익률과 시장 수익률 간의 회귀 분석
X_capm = market_data['Daily Return']
y_capm = stock_data['Daily Return'] - ff_factors['RF'] / 100

X_capm = sm.add_constant(X_capm)
capm_model = sm.OLS(y_capm, X_capm).fit()
print(capm_model.summary())

# Fama-French 3-팩터 모델: 주식 수익률과 팩터 수익률 간의 회귀 분석
ff_data = ff_factors.join(stock_data[['Daily Return']])
ff_data = ff_data.dropna()
X_ff = ff_data[['Mkt-RF', 'SMB', 'HML']]
y_ff = ff_data['Daily Return'] - ff_factors['RF'] / 100

X_ff = sm.add_constant(X_ff)
ff_model = sm.OLS(y_ff, X_ff).fit()
print(ff_model.summary())


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.610
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     1609.
Date:                Wed, 12 Jun 2024   Prob (F-statistic):          1.48e-212
Time:                        02:46:09   Log-Likelihood:                 3282.2
No. Observations:                1031   AIC:                            -6560.
Df Residuals:                    1029   BIC:                            -6551.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         9.244e-05      0.000      0.296   

In [6]:
!pip install yfinance
!pip install pandas
!pip install numpy
!pip install statsmodels

import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime

sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()

low_price_stocks = []
start_date = '2023-01-02'
end_date = datetime.today().strftime('%Y-%m-%d')

for ticker in sp500_tickers:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        if hist['Close'][-1] >= 1 and hist['Close'][-1] <= 15:
            low_price_stocks.append(ticker)
    except Exception as e:
        continue

print(f"Stocks between 1 and 15 dollars: {low_price_stocks}")

results = []

# 팩터 데이터 다운로드 (Fama-French 3-팩터)
url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip'
response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open('F-F_Research_Data_Factors_daily.CSV') as f:
        ff_factors = pd.read_csv(f, skiprows=3)

ff_factors = ff_factors.iloc[:-1]
ff_factors['Date'] = pd.to_datetime(ff_factors['Unnamed: 0'], format='%Y%m%d')
ff_factors.set_index('Date', inplace=True)
ff_factors = ff_factors[['Mkt-RF', 'SMB', 'HML', 'RF']].astype(float)

# 시장 데이터 (S&P 500 지수)
market_data = yf.download('^GSPC', start=start_date, end=end_date)
market_data['Daily Return'] = market_data['Adj Close'].pct_change()

for ticker in low_price_stocks:
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        stock_data['Daily Return'] = stock_data['Adj Close'].pct_change()
        stock_data = stock_data.dropna()

        common_index = stock_data.index.intersection(market_data.index).intersection(ff_factors.index)
        stock_data = stock_data.loc[common_index]
        market_data = market_data.loc[common_index]
        ff_factors = ff_factors.loc[common_index]

        # CAPM
        X_capm = market_data['Daily Return']
        y_capm = stock_data['Daily Return'] - ff_factors['RF'] / 100
        X_capm = sm.add_constant(X_capm)
        capm_model = sm.OLS(y_capm, X_capm).fit()

        # Fama-French 3-팩터 모델
        ff_data = ff_factors.join(stock_data[['Daily Return']])
        ff_data = ff_data.dropna()
        X_ff = ff_data[['Mkt-RF', 'SMB', 'HML']]
        y_ff = ff_data['Daily Return'] - ff_factors['RF'] / 100
        X_ff = sm.add_constant(X_ff)
        ff_model = sm.OLS(y_ff, X_ff).fit()

        # 분석 결과 저장
        results.append({
            'Symbol': ticker,
            'CAPM R-squared': capm_model.rsquared,
            'Fama-French R-squared': ff_model.rsquared
        })
    except Exception as e:
        continue

# R-squared 값을 기준으로 상위 10개 종목 선택
results = sorted(results, key=lambda x: (x['CAPM R-squared'], x['Fama-French R-squared']), reverse=True)[:10]
print(results)

selected_stocks = [result['Symbol'] for result in results]

stock_info = []

for ticker in selected_stocks:
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start_date, end=end_date)
    current_price = hist['Close'][-1]
    recent_close = hist['Close'][-2]

    # 간단한 매매 전략 설정
    trend = 'Uptrend' if current_price > recent_close else 'Downtrend'
    buy_range = (current_price * 0.95, current_price * 1.05)
    first_buy = current_price * 0.97
    second_buy = current_price * 0.95
    third_buy = current_price * 0.93
    take_profit = current_price * 1.10
    stop_loss = current_price * 0.90
    avg_volume = hist['Volume'].mean()
    volatility = hist['Close'].pct_change().std()

    stock_info.append({
        'Symbol': ticker,
        'Current Price': current_price,
        'Recent Close': recent_close,
        'Trend': trend,
        'Buy Range': buy_range,
        'First Buy': first_buy,
        'Second Buy': second_buy,
        'Third Buy': third_buy,
        'Take Profit': take_profit,
        'Stop Loss': stop_loss,
        'Avg Volume': avg_volume,
        'Volatility': volatility
    })

for info in stock_info:
    print(info)



ERROR:yfinance:$BRK.B: possibly delisted; No timezone found
ERROR:yfinance:$BF.B: possibly delisted; No price data found  (1d 2023-01-02 -> 2024-06-12)


$BF.B: possibly delisted; No price data found  (1d 2023-01-02 -> 2024-06-12)
Stocks between 1 and 15 dollars: ['AMCR', 'AAL', 'F', 'HBAN', 'IVZ', 'KEY', 'PARA', 'VTRS', 'WBD']


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[{'Symbol': 'IVZ', 'CAPM R-squared': 0.31902720009203966, 'Fama-French R-squared': 0.5417924712191856}, {'Symbol': 'AMCR', 'CAPM R-squared': 0.2832025949220006, 'Fama-French R-squared': 0.3561685188654191}, {'Symbol': 'F', 'CAPM R-squared': 0.2664022226167273, 'Fama-French R-squared': 0.3492053978167764}, {'Symbol': 'HBAN', 'CAPM R-squared': 0.2510311899072306, 'Fama-French R-squared': 0.6708015347091831}, {'Symbol': 'KEY', 'CAPM R-squared': 0.2165150600083856, 'Fama-French R-squared': 0.6256008379589713}, {'Symbol': 'AAL', 'CAPM R-squared': 0.21251950532332087, 'Fama-French R-squared': 0.29593387059316034}, {'Symbol': 'WBD', 'CAPM R-squared': 0.18910596595195128, 'Fama-French R-squared': 0.3394887805351362}, {'Symbol': 'VTRS', 'CAPM R-squared': 0.1351863669829455, 'Fama-French R-squared': 0.27831686925389254}, {'Symbol': 'PARA', 'CAPM R-squared': 0.12064658593264677, 'Fama-French R-squared': 0.25621427747906456}]
{'Symbol': 'IVZ', 'Current Price': 14.979999542236328, 'Recent Close': 1

In [8]:
!pip install yfinance
!pip install pandas
!pip install numpy
!pip install statsmodels

import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
import requests
import zipfile
import io

sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()

low_price_stocks = []
start_date = '2023-01-02'
end_date = datetime.today().strftime('%Y-%m-%d')

for ticker in sp500_tickers:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        if hist['Close'][-1] >= 1 and hist['Close'][-1] <= 15:
            low_price_stocks.append(ticker)
    except Exception as e:
        continue

print(f"Stocks between 1 and 15 dollars: {low_price_stocks}")

results = []

# 팩터 데이터 다운로드 (Fama-French 3-팩터)
url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip'
response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open('F-F_Research_Data_Factors_daily.CSV') as f:
        ff_factors = pd.read_csv(f, skiprows=3)

ff_factors = ff_factors.iloc[:-1]
ff_factors['Date'] = pd.to_datetime(ff_factors['Unnamed: 0'], format='%Y%m%d')
ff_factors.set_index('Date', inplace=True)
ff_factors = ff_factors[['Mkt-RF', 'SMB', 'HML', 'RF']].astype(float)

# 시장 데이터 (S&P 500 지수)
market_data = yf.download('^GSPC', start=start_date, end=end_date)
market_data['Daily Return'] = market_data['Adj Close'].pct_change()

for ticker in low_price_stocks:
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        stock_data['Daily Return'] = stock_data['Adj Close'].pct_change()
        stock_data = stock_data.dropna()

        common_index = stock_data.index.intersection(market_data.index).intersection(ff_factors.index)
        stock_data = stock_data.loc[common_index]
        market_data = market_data.loc[common_index]
        ff_factors = ff_factors.loc[common_index]

        # CAPM
        X_capm = market_data['Daily Return']
        y_capm = stock_data['Daily Return'] - ff_factors['RF'] / 100
        X_capm = sm.add_constant(X_capm)
        capm_model = sm.OLS(y_capm, X_capm).fit()

        # Fama-French 3-팩터 모델
        ff_data = ff_factors.join(stock_data[['Daily Return']])
        ff_data = ff_data.dropna()
        X_ff = ff_data[['Mkt-RF', 'SMB', 'HML']]
        y_ff = ff_data['Daily Return'] - ff_factors['RF'] / 100
        X_ff = sm.add_constant(X_ff)
        ff_model = sm.OLS(y_ff, X_ff).fit()

        # 분석 결과 저장
        results.append({
            'Symbol': ticker,
            'CAPM R-squared': capm_model.rsquared,
            'Fama-French R-squared': ff_model.rsquared
        })
    except Exception as e:
        continue

# R-squared 값을 기준으로 상위 10개 종목 선택
results = sorted(results, key=lambda x: (x['CAPM R-squared'], x['Fama-French R-squared']), reverse=True)[:10]
top_10_stocks_df = pd.DataFrame(results)
print("조건에 맞는 상위 10개 종목 목록:")
print(top_10_stocks_df)

stock_info = []

for ticker in top_10_stocks_df['Symbol']:
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start_date, end=end_date)
    current_price = hist['Close'][-1]
    recent_close = hist['Close'][-2]

    # 간단한 매매 전략 설정
    trend = 'Uptrend' if current_price > recent_close else 'Downtrend'
    buy_range = (current_price * 0.95, current_price * 1.05)
    first_buy = current_price * 0.97
    second_buy = current_price * 0.95
    third_buy = current_price * 0.93
    take_profit = current_price * 1.10
    stop_loss = current_price * 0.90
    avg_volume = hist['Volume'].mean()
    volatility = hist['Close'].pct_change().std()

    stock_info.append({
        'Symbol': ticker,
        'Current Price': current_price,
        'Recent Close': recent_close,
        'Trend': trend,
        'Buy Range': buy_range,
        'First Buy': first_buy,
        'Second Buy': second_buy,
        'Third Buy': third_buy,
        'Take Profit': take_profit,
        'Stop Loss': stop_loss,
        'Avg Volume': avg_volume,
        'Volatility': volatility
    })

stock_info_df = pd.DataFrame(stock_info)
print("선정된 종목의 상세 정보:")
print(stock_info_df)



ERROR:yfinance:$BRK.B: possibly delisted; No timezone found
ERROR:yfinance:$BF.B: possibly delisted; No price data found  (1d 2023-01-02 -> 2024-06-12)


$BF.B: possibly delisted; No price data found  (1d 2023-01-02 -> 2024-06-12)
Stocks between 1 and 15 dollars: ['AMCR', 'AAL', 'F', 'HBAN', 'IVZ', 'KEY', 'PARA', 'VTRS', 'WBD']


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


조건에 맞는 상위 10개 종목 목록:
  Symbol  CAPM R-squared  Fama-French R-squared
0    IVZ        0.319027               0.541793
1   AMCR        0.283202               0.356168
2      F        0.266402               0.349206
3   HBAN        0.251031               0.670801
4    KEY        0.216515               0.625601
5    AAL        0.212520               0.295934
6    WBD        0.189106               0.339489
7   VTRS        0.135187               0.278317
8   PARA        0.120647               0.256214
선정된 종목의 상세 정보:
  Symbol  Current Price  Recent Close      Trend  \
0    IVZ          14.98         15.19  Downtrend   
1   AMCR           9.98          9.91    Uptrend   
2      F          12.11         12.38  Downtrend   
3   HBAN          12.35         12.58  Downtrend   
4    KEY          13.42         13.67  Downtrend   
5    AAL          11.28         11.49  Downtrend   
6    WBD           8.04          8.12  Downtrend   
7   VTRS          10.64         10.51    Uptrend   
8   PARA        

In [9]:
!pip install yfinance
!pip install pandas
!pip install numpy
!pip install statsmodels

import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
import requests
import zipfile
import io
from IPython.display import display, HTML

sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()

low_price_stocks = []
start_date = '2023-01-02'
end_date = datetime.today().strftime('%Y-%m-%d')

for ticker in sp500_tickers:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        if hist['Close'][-1] >= 1 and hist['Close'][-1] <= 15:
            low_price_stocks.append(ticker)
    except Exception as e:
        continue

print(f"Stocks between 1 and 15 dollars: {low_price_stocks}")

results = []

# 팩터 데이터 다운로드 (Fama-French 3-팩터)
url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip'
response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open('F-F_Research_Data_Factors_daily.CSV') as f:
        ff_factors = pd.read_csv(f, skiprows=3)

ff_factors = ff_factors.iloc[:-1]
ff_factors['Date'] = pd.to_datetime(ff_factors['Unnamed: 0'], format='%Y%m%d')
ff_factors.set_index('Date', inplace=True)
ff_factors = ff_factors[['Mkt-RF', 'SMB', 'HML', 'RF']].astype(float)

# 시장 데이터 (S&P 500 지수)
market_data = yf.download('^GSPC', start=start_date, end=end_date)
market_data['Daily Return'] = market_data['Adj Close'].pct_change()

for ticker in low_price_stocks:
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        stock_data['Daily Return'] = stock_data['Adj Close'].pct_change()
        stock_data = stock_data.dropna()

        common_index = stock_data.index.intersection(market_data.index).intersection(ff_factors.index)
        stock_data = stock_data.loc[common_index]
        market_data = market_data.loc[common_index]
        ff_factors = ff_factors.loc[common_index]

        # CAPM
        X_capm = market_data['Daily Return']
        y_capm = stock_data['Daily Return'] - ff_factors['RF'] / 100
        X_capm = sm.add_constant(X_capm)
        capm_model = sm.OLS(y_capm, X_capm).fit()

        # Fama-French 3-팩터 모델
        ff_data = ff_factors.join(stock_data[['Daily Return']])
        ff_data = ff_data.dropna()
        X_ff = ff_data[['Mkt-RF', 'SMB', 'HML']]
        y_ff = ff_data['Daily Return'] - ff_factors['RF'] / 100
        X_ff = sm.add_constant(X_ff)
        ff_model = sm.OLS(y_ff, X_ff).fit()

        # 분석 결과 저장
        results.append({
            'Symbol': ticker,
            'CAPM R-squared': capm_model.rsquared,
            'Fama-French R-squared': ff_model.rsquared
        })
    except Exception as e:
        continue

# R-squared 값을 기준으로 상위 10개 종목 선택
results = sorted(results, key=lambda x: (x['CAPM R-squared'], x['Fama-French R-squared']), reverse=True)[:10]
top_10_stocks_df = pd.DataFrame(results)
print("조건에 맞는 상위 10개 종목 목록:")
print(top_10_stocks_df)

# 필터링된 종목 리스트 데이터프레임 출력 함수
def display_dataframe_to_user(name: str, dataframe: pd.DataFrame) -> None:
    display(HTML(f"<h2>{name}</h2>"))
    display(dataframe)

display_dataframe_to_user(name="조건에 맞는 상위 10개 S&P500 종목 목록", dataframe=top_10_stocks_df)

results = []

# 팩터 데이터 다운로드 (Fama-French 3-팩터)
url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip'
response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open('F-F_Research_Data_Factors_daily.CSV') as f:
        ff_factors = pd.read_csv(f, skiprows=3)

ff_factors = ff_factors.iloc[:-1]
ff_factors['Date'] = pd.to_datetime(ff_factors['Unnamed: 0'], format='%Y%m%d')
ff_factors.set_index('Date', inplace=True)
ff_factors = ff_factors[['Mkt-RF', 'SMB', 'HML', 'RF']].astype(float)

# 시장 데이터 (S&P 500 지수)
market_data = yf.download('^GSPC', start=start_date, end=end_date)
market_data['Daily Return'] = market_data['Adj Close'].pct_change()

for ticker in low_price_stocks:
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        stock_data['Daily Return'] = stock_data['Adj Close'].pct_change()
        stock_data = stock_data.dropna()

        common_index = stock_data.index.intersection(market_data.index).intersection(ff_factors.index)
        stock_data = stock_data.loc[common_index]
        market_data = market_data.loc[common_index]
        ff_factors = ff_factors.loc[common_index]

        # CAPM
        X_capm = market_data['Daily Return']
        y_capm = stock_data['Daily Return'] - ff_factors['RF'] / 100
        X_capm = sm.add_constant(X_capm)
        capm_model = sm.OLS(y_capm, X_capm).fit()

        # Fama-French 3-팩터 모델
        ff_data = ff_factors.join(stock_data[['Daily Return']])
        ff_data = ff_data.dropna()
        X_ff = ff_data[['Mkt-RF', 'SMB', 'HML']]
        y_ff = ff_data['Daily Return'] - ff_factors['RF'] / 100
        X_ff = sm.add_constant(X_ff)
        ff_model = sm.OLS(y_ff, X_ff).fit()

        # 분석 결과 저장
        results.append({
            'Symbol': ticker,
            'CAPM R-squared': capm_model.rsquared,
            'Fama-French R-squared': ff_model.rsquared
        })
    except Exception as e:
        continue

# R-squared 값을 기준으로 상위 10개 종목 선택
results = sorted(results, key=lambda x: (x['CAPM R-squared'], x['Fama-French R-squared']), reverse=True)[:10]
top_10_stocks_df = pd.DataFrame(results)
print("조건에 맞는 상위 10개 종목 목록:")
print(top_10_stocks_df)

# 필터링된 종목 리스트 데이터프레임 출력 함수
def display_dataframe_to_user(name: str, dataframe: pd.DataFrame) -> None:
    display(HTML(f"<h2>{name}</h2>"))
    display(dataframe)

display_dataframe_to_user(name="조건에 맞는 상위 10개 S&P500 종목 목록", dataframe=top_10_stocks_df)

stock_info = []

for ticker in top_10_stocks_df['Symbol']:
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start_date, end=end_date)
    current_price = hist['Close'][-1]
    recent_close = hist['Close'][-2]

    # 간단한 매매 전략 설정
    trend = 'Uptrend' if current_price > recent_close else 'Downtrend'
    buy_range = (current_price * 0.95, current_price * 1.05)
    first_buy = current_price * 0.97
    second_buy = current_price * 0.95
    third_buy = current_price * 0.93
    take_profit = current_price * 1.10
    stop_loss = current_price * 0.90
    avg_volume = hist['Volume'].mean()
    volatility = hist['Close'].pct_change().std()

    stock_info.append({
        'Symbol': ticker,
        'Current Price': current_price,
        'Recent Close': recent_close,
        'Trend': trend,
        'Buy Range': buy_range,
        'First Buy': first_buy,
        'Second Buy': second_buy,
        'Third Buy': third_buy,
        'Take Profit': take_profit,
        'Stop Loss': stop_loss,
        'Avg Volume': avg_volume,
        'Volatility': volatility
    })

stock_info_df = pd.DataFrame(stock_info)
print("선정된 종목의 상세 정보:")
print(stock_info_df)

display_dataframe_to_user(name="선정된 종목의 상세 정보", dataframe=stock_info_df)


ERROR:yfinance:$BRK.B: possibly delisted; No timezone found
ERROR:yfinance:$BF.B: possibly delisted; No price data found  (1d 2023-01-02 -> 2024-06-12)


$BF.B: possibly delisted; No price data found  (1d 2023-01-02 -> 2024-06-12)
Stocks between 1 and 15 dollars: ['AMCR', 'AAL', 'F', 'HBAN', 'IVZ', 'KEY', 'PARA', 'VTRS', 'WBD']


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


조건에 맞는 상위 10개 종목 목록:
  Symbol  CAPM R-squared  Fama-French R-squared
0    IVZ        0.319028               0.541792
1   AMCR        0.283202               0.356168
2      F        0.266403               0.349206
3   HBAN        0.251031               0.670801
4    KEY        0.216515               0.625601
5    AAL        0.212520               0.295934
6    WBD        0.189106               0.339489
7   VTRS        0.135187               0.278317
8   PARA        0.120647               0.256214


,Symbol,CAPM R-squared,Fama-French R-squared
0,IVZ,0.319028,0.541792
1,AMCR,0.283202,0.356168
2,F,0.266403,0.349206
3,HBAN,0.251031,0.670801
4,KEY,0.216515,0.625601
5,AAL,0.212520,0.295934
6,WBD,0.189106,0.339489
7,VTRS,0.135187,0.278317
8,PARA,0.120647,0.256214


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

조건에 맞는 상위 10개 종목 목록:
  Symbol  CAPM R-squared  Fama-French R-squared
0    IVZ        0.319028               0.541792
1   AMCR        0.283202               0.356168
2      F        0.266403               0.349206
3   HBAN        0.251031               0.670801
4    KEY        0.216515               0.625601
5    AAL        0.212520               0.295934
6    WBD        0.189106               0.339489
7   VTRS        0.135187               0.278317
8   PARA        0.120647               0.256214


,Symbol,CAPM R-squared,Fama-French R-squared
0,IVZ,0.319028,0.541792
1,AMCR,0.283202,0.356168
2,F,0.266403,0.349206
3,HBAN,0.251031,0.670801
4,KEY,0.216515,0.625601
5,AAL,0.212520,0.295934
6,WBD,0.189106,0.339489
7,VTRS,0.135187,0.278317
8,PARA,0.120647,0.256214


선정된 종목의 상세 정보:
  Symbol  Current Price  Recent Close      Trend  \
0    IVZ          14.98         15.19  Downtrend   
1   AMCR           9.98          9.91    Uptrend   
2      F          12.11         12.38  Downtrend   
3   HBAN          12.35         12.58  Downtrend   
4    KEY          13.42         13.67  Downtrend   
5    AAL          11.28         11.49  Downtrend   
6    WBD           8.04          8.12  Downtrend   
7   VTRS          10.64         10.51    Uptrend   
8   PARA          11.04         11.98  Downtrend   

                                  Buy Range  First Buy  Second Buy  Third Buy  \
0  (14.230999565124511, 15.728999519348145)    14.5306     14.2310    13.9314   
1   (9.480999565124511, 10.478999519348145)     9.6806      9.4810     9.2814   
2  (11.504499673843384, 12.715499639511108)    11.7467     11.5045    11.2623   
3   (11.73250036239624, 12.967500400543214)    11.9795     11.7325    11.4855   
4  (12.749000072479248, 14.091000080108643)    13.0174     

,Symbol,Current Price,Recent Close,Trend,Buy Range,First Buy,Second Buy,Third Buy,Take Profit,Stop Loss,Avg Volume,Volatility
0,IVZ,14.98,15.19,Downtrend,"(14.230999565124511, 15.728999519348145)",14.5306,14.2310,13.9314,16.477999,13.482,4.758240e+06,0.019958
1,AMCR,9.98,9.91,Uptrend,"(9.480999565124511, 10.478999519348145)",9.6806,9.4810,9.2814,10.977999,8.982,8.077600e+06,0.013999
2,F,12.11,12.38,Downtrend,"(11.504499673843384, 12.715499639511108)",11.7467,11.5045,11.2623,13.321000,10.899,5.607729e+07,0.021627
3,HBAN,12.35,12.58,Downtrend,"(11.73250036239624, 12.967500400543214)",11.9795,11.7325,11.4855,13.585000,11.115,1.609819e+07,0.021055
4,KEY,13.42,13.67,Downtrend,"(12.749000072479248, 14.091000080108643)",13.0174,12.7490,12.4806,14.762000,12.078,1.746393e+07,0.031372
5,AAL,11.28,11.49,Downtrend,"(10.715999746322632, 11.84399971961975)",10.9416,10.7160,10.4904,12.408000,10.152,2.982292e+07,0.024636
6,WBD,8.04,8.12,Downtrend,"(7.637999963760375, 8.44199995994568)",7.7988,7.6380,7.4772,8.844000,7.236,2.309125e+07,0.032053
7,VTRS,10.64,10.51,Uptrend,"(10.108000326156615, 11.172000360488893)",10.3208,10.1080,9.8952,11.704000,9.576,9.080129e+06,0.016326
8,PARA,11.04,11.98,Downtrend,"(10.487999963760375, 11.59199995994568)",10.7088,10.4880,10.2672,12.144000,9.936,1.507554e+07,0.038581
